<a href="https://colab.research.google.com/github/busrakurunceng/phi4-finetuning/blob/main/Phi_4_Mini_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📄 Fine-tuning Phi-4 Mini Instruct

 Fine-tuning Phi-4 Mini Instruct Model

 This notebook demonstrates how to fine-tune the `microsoft/phi-4-mini-instruct` model using the Hugging Face `transformers` library.

 We will:
 - Load the model and tokenizer
 - Prepare a small dataset
 - Fine-tune the model
 - Save the fine-tuned model
 - Log the training loss in a clean tabular format

---

## 1. Install Required Libraries and Import Libraries

In [ ]:
!pip install transformers datasets accelerate peft trl bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import TrainerCallback
from tabulate import tabulate

Download the model and tokenizer from Hugging Face.



In [ ]:

model_id = "microsoft/Phi-4-mini-instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    load_in_4bit=True,  # 4-bit modunda yükle
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    trust_remote_code=True
)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## 2.Install Model and Tokenizer

In [ ]:

# Modeli k-bit eğitime hazırla (4-bit olduğundan şart)
model = prepare_model_for_kbit_training(model)

# LoRA ayarları
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["self_attn.qkv_proj", "self_attn.o_proj", "mlp.gate_up_proj", "mlp.down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)


model = get_peft_model(model, lora_config)

## 3. Make Dataset

In [ ]:
# Örnek kendi veri kümesi
sample_data = [
    {
        "messages": [
            {"role": "user", "content": "What is machine learning?"},
            {"role": "assistant", "content": "Machine learning is a field of AI focused on building systems that learn from data."}
        ]
    },
    {
        "messages": [
            {"role": "user", "content": "Explain natural language processing."},
            {"role": "assistant", "content": "Natural language processing involves teaching machines to understand human language."}
        ]
    }
]

import json

# JSONL dosyasına kaydet
with open('train.jsonl', 'w') as f:
    for item in sample_data:
        f.write(json.dumps(item) + "\n")

# Dataseti yükle
dataset = load_dataset("json", data_files="train.jsonl")["train"]


Generating train split: 0 examples [00:00, ? examples/s]

## 4. Tokenize Dataset

In [ ]:
def tokenize_function(example):
    user_message = example['messages'][0]['content']
    assistant_message = example['messages'][1]['content']
    prompt = f"<|user|>\n{user_message}\n<|assistant|>\n{assistant_message}"
    result = tokenizer(prompt, truncation=True, padding="max_length", max_length=512)
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_dataset = dataset.map(tokenize_function, batched=False)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

## 5. Define Custom Logging Callback
This callback formats the loss output in a clean table every logging step.

In [ ]:
class LoggingCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            table = [
                ["Step", state.global_step],
                ["Training Loss", round(logs.get("loss", 0.0), 4)]
            ]
            print(tabulate(table, headers=["Metric", "Value"], tablefmt="fancy_grid"))

## 6. Define Training Arguments

Set up training parameters: batch size, epochs, logging, and precision settings. Prepare the Hugging Face Trainer with the model, training arguments, and the dataset.


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./phi4-finetuned",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    num_train_epochs=2,
    logging_steps=20,
    save_steps=100,
    save_total_limit=1,
    fp16=True,
    report_to="none",
    disable_tqdm=True  # TQDM progress bar kapalı çünkü tablo basacağız
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    callbacks=[LoggingCallback()]
)



<ipython-input-16-8d0df0b58964>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


## 7. Start Training
Fine-tune the model on the prepared dataset!



In [ ]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended,

╒═══════════════╤═════════╕
│ Metric        │   Value │
╞═══════════════╪═════════╡
│ Step          │       2 │
├───────────────┼─────────┤
│ Training Loss │       0 │
╘═══════════════╧═════════╛
{'train_runtime': 6.5117, 'train_samples_per_second': 0.614, 'train_steps_per_second': 0.307, 'train_loss': 11.137123107910156, 'epoch': 2.0}


TrainOutput(global_step=2, training_loss=11.137123107910156, metrics={'train_runtime': 6.5117, 'train_samples_per_second': 0.614, 'train_steps_per_second': 0.307, 'train_loss': 11.137123107910156, 'epoch': 2.0})

## 8. Save the Fine-tuned Model
Save the fine-tuned model and tokenizer for future use.

In [ ]:
trainer.save_model("./phi4-finetuned-final")
tokenizer.save_pretrained("./phi4-finetuned-final")


('./phi4-finetuned-final/tokenizer_config.json',
 './phi4-finetuned-final/special_tokens_map.json',
 './phi4-finetuned-final/vocab.json',
 './phi4-finetuned-final/merges.txt',
 './phi4-finetuned-final/added_tokens.json',
 './phi4-finetuned-final/tokenizer.json')